In [10]:
#########################################################
# 구현해야 하는 기본 모듈: map / patient / hospital

#########################################################
######################## For MAP ########################
#########################################################
"""
[ Attributes] 

1. GRID:
	- min_lat								(float)
    - max_lat								(float)
    - min_lon								(float)
    - max_lon								(float)
2. districts: 								(dict)
	{0:[center_location, radius], ...}
    - 0: 관악구
    - center_location: mu_1, mu_2 			(tuple)
    - radius: np.sqrt(size/np.pi) 			(float)
3. weights: 								(list)
	[관악구 인구 %, 그 외 다른 거 %, ]
4. patient_dict:							(dict)
	{id: class Patient(), ...}
5. acc_patient_num:							(int)

[ Methods ]

1. create_patient(self, no arg)				(func)
	- id = self.acc_patient_num
    - d_id = np.random.choice(self.weights)
      lat, lon = np.random.multivariatenormal(self.districts[d_id][0], self.districts[d_id][0])
      lat = max(min(lat, max_lat), min_lat)
      lon = max(min(lon, max_lon), min_lon)
      location = (lat, lon)
    - created_time = tau
    - state = 0
    - destiny = np.random."oracle"
      (oracle: 처음에는 사망 확률이 낮다가, 시간 지나면 상승, left skewed)			[GPT 부탁해!]
    
    (그래서)
    self.patient_dict[id] = Patient(id, location, created_time, state, destiny)
    
    return
"""







#########################################################
# 
import numpy as np

if __name__ == '__main__':
    with open('../config/config.json') as f:
        config = json.load(f)
        
    min_lat = config['min_lat']
    max_lat = config['max_lat']
    min_lon = config['min_lon']
    max_lon = config['max_lon']
    districts = config['districts'] # dict; {0: [(lat, lon), radius], ...}
    weights = config['weights'] # list; population percentages
    hospitals = config['hospitals'] # dict; {0: (lat, lon), ...}
    mean_treat_times = config['mean_treat_times'] # list; average treatment time in each hospital
        
    GRID = {'min_lat': min_lat, 'max_lat': max_lat, 'min_lon': min_lon, 'max_lon': max_lon}
    patient_dict = {}  # dict; To store Patient instances
    acc_patient_num = 0 # int; accumulated patient number
    
    tau = 0 # Simulation global time step
    end_time = 24 * 3600
    
    # Initialize the time points when patients occur on the map
    patient_generate_mean = 1200
    
    patient_generate_points = []
    time_point = 0
    
    while time_point < end_time:
      sample = np.random.exponential(1200, 1)
      time_point += sample
      
      if time_point < end_time:
        patient_generate_points.append(time_point)
    

    hospitals_dict = {}
    for id in range(len(hospitals)):
      hospitals_dict[id] = hospital(id, hospitals[id], mean_treat_times[id])
      
    while tau <= end_time:
      if tau in patient_generate_points:
        create_patient()
      updater()
      tau += 1
        
      
      
      

def create_patient():
    global tau, acc_patient_num, weights, districts, GRID 
    
    id = acc_patient_num
    d_id = np.random.choice(len(weights), p=weights)
    mean = districts[d_id][0]
    cov = [[districts[d_id][1]**2, 0], [0, districts[d_id][1]**2]]
    
    lat, lon = np.random.multivariate_normal(mean, cov)
    lat = max(min(lat, GRID['max_lat']), GRID['min_lat'])
    lon = max(min(lon, GRID['max_lon']), GRID['min_lon'])
    location = (lat, lon)

    created_time = tau # Global time step
    state = 0 # 0: Alive
    
    alpha_skewed = 8 # Standard Alpha
    beta_skewed = 3 # Standard Beta 
    a_skewed = 0 # Min 생존 시간
    b_skewed = 3600 # Max 생존 시간
    
    # skew 고려해서 standard beta 설정 (by alpha_skewed, beta_skewed)    
    destiny_temp = np.random.beta(alpha_skewed, beta_skewed, 1)
                   
    # domain 늘이기 (from a_skewed to b_skewed)
    destiny = a_skewed + (b_skewed - a_skewed) * destiny_temp

    # Assuming Patient class is defined elsewhere
    patient_dict[id] = Patient(id, location, created_time, state, destiny)
    
    acc_patient_num += 1

class Patient(self):
  def __init__(self, id:int, location:tuple, created_time:int, state:int, destiny:float):
    self.id = id
    self.location = location   #tuple (float, float)
    self.created_time = created_time
    self.state = state  # 0: 이송중 1: 큐 대기중 2: 사망
    self.destiny = destiny

    
class Hospital(self):
  def __init__(self, hospitals_id:int, hospitals_location:tuple, mean_treat_time:int):
    self.queue = queue.Queue()
    self.len_queue = 0
    self.treat_start = None
    self.id = hospitals_id
    self.location = hospitals_location
    self.mean_treat_time = mean_treat_time
  
  def sample_treat_time(self):
    treat_time = np.random.exponential(self.mean_treat_time, 1)
    return treat_time

def haversine(lat1, lon1, lat2, lon2):
  """
  Calculate the great circle distance in kilometers between two points 
  on the earth (specified in decimal degrees)
  """
  # convert decimal degrees to radians 
  lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

  # haversine formula 
  dlat = lat2 - lat1 
  dlon = lon2 - lon1 
  a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
  c = 2 * asin(sqrt(a)) 
  r = 6371 # Radius of earth in kilometers. Use 3956 for miles
  return c * r

def compute_etaw(patient: Patient, hospital: Hospital, tau: int):
  patient_lat, patient_lon = patient.location
  hospital_lat, hospital_lon = hospital.location

  # Calculate the real distance using Haversine formula
  distance = haversine(patient_lat, patient_lon, hospital_lat, hospital_lon)  ## km
  velocity = 30
  eta = distance / velocity * 3600  
  wait = (hospital.len_queue + 1 + np.sqrt(hospital.len_queue + 1)) * hospital.mean_treat_time
  etaw = eta + wait
    
  return eta, etaw

def updater():
  ...

2

In [2]:
print("Yes")

Yes


In [3]:
import queue
a = queue.Queue()
a.put(1)
a.put(2)
a.put(3)
a.put(4)
a.put(5)
del a.queue[2]
a.queue

deque([1, 2, 4, 5])